# Imports

In [1]:
# to prevent to restart kernel when any changes are made to any imported file
%reload_ext autoreload
%autoreload 2

# to import any file from some other directory
# sys.path.append("/tmp/fastai/old")

# to stop printing warnings
import warnings
warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
warnings.warn = warn
    
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_colwidth', -1)

# to increase cells width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# to enable collapsible Headings and Functions
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter nbextensions_configurator enable --user
# !jupyter nbextension enable codefolding/main
# search collapsible to enable

# enable dark theme
# !pip install jupyterthemes
# !jt -t monokai

# monokai
# solarizedd
# !jt -r
import numpy as np
import faiss
from faiss import normalize_L2
import pandas as pd
import sys, os
from datetime import datetime, timedelta
import requests
import json
import timeit
from datetime import datetime
import os.path, time
from datetime import datetime, timedelta
from threading import Timer
import pdb, random

Loading faiss.


In [2]:
from get_glove_emb import get_word2vec_embeddings



Found 209 word vectors. 1


# Get Data

In [3]:
from random import randint


PATH="../real_not_real_kaggle/data/"
train=pd.read_csv(f"{PATH}train.csv").reset_index(drop=True)
test=pd.read_csv(f"{PATH}test.csv").reset_index(drop=True)
print(train.shape, test.shape)

# train["key"] = ["KEY"+x for x in list(map(str, random.sample(range(1000000, 9999999), len(train))))]
# test["key"]  = ["KEY"+x for x in list(map(str, random.sample(range(1000000, 9999999), len(test))))]
train["key"] = "KEY"+train.index.astype(str)
test["key"]  = "KEY"+test.index.astype(str)

# tr_embeddings = get_word2vec_embeddings(train, "tokens")
# ts_embeddings = get_word2vec_embeddings(test, "tokens")
# train["emb"]=list(tr_embeddings)
# test["emb"]=list(ts_embeddings)

# print(np.array(tr_embeddings).shape)
# print(np.array(ts_embeddings).shape)



(7613, 5) (3263, 4)


In [4]:
# test["key"].sample(200)
train.head()

,id,keyword,location,text,target,key
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,KEY0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,KEY1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,KEY2
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,KEY3
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,KEY4


In [5]:
test.shape, test.key.nunique()

((3263, 5), 3263)

In [6]:
# df=pd.read_pickle("/home/cloud_user/faisal/data/data_latest")
# df=df.rename(columns={"number": "key", "merged": "text"})
# print(df.shape)

In [7]:
def create_save_index(df, ids_col, xb_faiss_col, indx_fn, ids_fn):
    try:
        
        ids=np.array(df[ids_col].str.slice(start=3).tolist()).astype(int)
        xb_faiss = np.float32(np.asarray(df[xb_faiss_col].tolist()))

        d  = xb_faiss.shape[1] 
        print(d, type(d))

        normalize_L2(xb_faiss)

        index_faiss = faiss.IndexIDMap(faiss.IndexFlatIP(d))
        index_faiss.add_with_ids(xb_faiss, ids)

        print(index_faiss.is_trained)
        print(index_faiss.ntotal)

        faiss.write_index(index_faiss, indx_fn)
        np.save(ids_fn, ids)

        return True
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname =os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("Exception get_similar_tickets: ", e, exc_type, fname, exc_tb.tb_lineno)
        return False

In [8]:
import numpy as np
from scipy import spatial


def avg_feature_vector(sentence, num_features, word_index):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in word_index:
            n_words += 1
            feature_vec = np.add(feature_vec, word_index[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec





In [9]:
word_index=pd.read_pickle(f"{PATH}glove.840B.300d.pkl")

In [10]:
len(word_index.keys())

2196008

In [11]:
train["emb"] = train["text"].apply(lambda x: avg_feature_vector(x, 300, word_index)) 
test["emb"]  = test["text"].apply(lambda x: avg_feature_vector(x, 300, word_index)) 




In [12]:
train["emb"].shape

(7613,)

# Test Script

## Glove

In [13]:
from similar_git import get_similar_tickets



In [20]:
s=timeit.default_timer()
result_df = get_similar_tickets(train, test.sample(10), "key", "emb", k=5, threshold=0.95)
e=timeit.default_timer()
print("Overall total time: ", e-s)

starting operations
creating faiss index
FAISS index trained:  True
Total db vectors:  7613
starting calculations
query vector created
got results from FAISS
creating result dataframe
Overall total time:  0.2267482930328697


In [21]:
result_df

,key,query_text,NN1_text,NN1_score,NN1_number,NN2_text,NN2_score,NN2_number,NN3_text,NN3_score,NN3_number,NN4_text,NN4_score,NN4_number,NN5_text,NN5_score,NN5_number
0,KEY1935,@MichiganAutoLaw's @SteveGursten will discuss #autoaccidents &amp; so-called 'mild' #TBIs on his @JusticeDotOrg webinar. http://t.co/ZUM3mluCYS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KEY1966,.@38_degrees Hello. I have been inundated by people Tweeting me about some Unilever thing in India I know nothing about.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KEY1976,WATCH: Likely landslide victims denied assistance again http://t.co/BxiohtiC5X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KEY2413,Last Chance Animal Rescue has 3 new posts. http://t.co/qtEqA5kvpy #animalrescue | https://t.co/ec46LyQQc6,Last Chance Animal Rescue has 3 new posts. http://t.co/kIILdu8GpO #animalrescue | https://t.co/wUDlkq7ncx,1,KEY5660,Last Chance Animal Rescue has 3 new posts. http://t.co/f1tcbg1MKi #animalrescue | https://t.co/Is2iDC3UBJ,1,KEY5659,Last Chance Animal Rescue has 3 new posts. http://t.co/1EB2DaUYfn #animalrescue,0.957132,KEY5667,NaN,NaN,NaN,NaN,NaN,NaN
4,KEY158,Hope the Salvation Army at least gave you a tax receipt. https://t.co/Ir9c545rZy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,KEY168,Israeli police unable to solve the case of Duma arson attack http://t.co/WtZgXzaf7Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,KEY43,Please donate and spread the word! A training accident left the pole-vaulter Kira GrÌ_nberg a paraplegic http://t.co/6MpnyCl8PK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,KEY499,We dont really talk about the murder of the bombing campaigns in WWII because we won but murder it is. https://t.co/jsGwnY6kdJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,KEY947,#?? #?? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... http://t.co/ykLruIMIsD,#?? #???? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... http://t.co/hHWv0EUDFv,1,KEY2170,#?? #?? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... http://t.co/MRVXBZywd4,1,KEY2165,#??? #?? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... http://t.co/zxCORQ0A3a,1,KEY2182,#?? #???? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... http://t.co/oTsM38XMas,1,KEY2175,#?? #?? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... http://t.co/q1GlK8plUD,1,KEY2168
9,KEY3116,On 70th anniversary of #Hiroshima bomb it is important to learn from history and ban this weapon.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
